# Importing Dependancies

In [37]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

# Read in Dataset

In [2]:
# read data into dataframe
df = pd.read_csv("./OnlineNewsPopularity.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

# Preprocessing

### Split into data and target labels

In [3]:
y = df.iloc[:, 60]
x = df.iloc[:, 0:60]

### Independant Variables

In [4]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 60 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

### Dependant Variables

In [5]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 39644 entries, 0 to 39643
Series name:  shares
Non-Null Count  Dtype
--------------  -----
39644 non-null  int64
dtypes: int64(1)
memory usage: 309.8 KB


### Drop Unused Features

In [6]:
# drop unused features
vals = [0, 1, 4, 5, 6]
for i in range(13, 39):
    vals.append(i)

x = x.drop(x.columns[vals], axis = 1)
x.head()

,n_tokens_title,n_tokens_content,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,LDA_00,LDA_01,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,12.0,219.0,4.0,2.0,1.0,0.0,4.680365,5.0,0.500331,0.378279,...,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500
1,9.0,255.0,3.0,1.0,1.0,0.0,4.913725,4.0,0.799756,0.050047,...,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000
2,9.0,211.0,3.0,1.0,1.0,0.0,4.393365,6.0,0.217792,0.033334,...,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000
3,9.0,531.0,9.0,0.0,1.0,0.0,4.404896,7.0,0.028573,0.419300,...,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000
4,13.0,1072.0,19.0,19.0,20.0,0.0,4.682836,7.0,0.028633,0.028794,...,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364


### Shuffle Data

In [7]:
np.random.seed(42)

In [8]:
# convert to numpy objects
x = x.to_numpy()
y = y.to_numpy()

In [9]:
# shuffle
arr = np.arange(len(df))
np.random.shuffle(arr)
x = x[arr]
y = y[arr]

### Splitting Data

In [10]:
x_train = x[:1000].copy()
y_train = y[:1000].copy()
x_val = x[1000:2000].copy()
y_val = y[1000:2000].copy()
x_test = x[2000:3000].copy()
y_test = y[2000:3000].copy()

### Standardize Data

In [11]:
# first save the training mean and std
training_mean = np.mean(x_train, axis=0)
training_std = np.std(x_train, axis=0)

# first standardize the training data
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
# standardize the test and validation data using training mean and std
x_test = (x_test - training_mean) / training_std
x_val = (x_val - training_mean) / training_std

### Clamping Data
I'll clamp the shares attribute to be at most 6000.

In [12]:
for i in range(1000):
    if y_train[i] > 6000:
        y_train[i] = 6000
    if y_test[i] > 6000:
        y_test[i] = 6000
    if y_val[i] > 6000:
        y_val[i] = 6000

### Map target data into two distinct classes
+1 is shares >= 2000 and -1 is shares < 2000

In [14]:
for i in range(1000):
    if y_train[i] >= 2000:
        y_train[i] = 1
    else:
        y_train[i] = -1
        
    if y_test[i] >= 2000:
        y_test[i] = 1
    else:
        y_test[i] = -1
        
    if y_val[i] >= 2000:
        y_val[i] = 1
    else:
        y_val[i] = -1

# Computing the Kernel Matrices
I will compute the linear and gaussian kernel and use each one to implement the Dual SVM algorithm  

### Linear Kernel

In [18]:
D = x_train.copy()

In [19]:
linear_kernel = np.dot(D, D.T)

In [20]:
linear_kernel

array([[23.56861317, -1.78768508, -9.04492404, ...,  8.87583137,
         6.88528597,  3.20230281],
       [-1.78768508, 12.57868004,  1.10172574, ..., -6.69551931,
        -2.99318777, -0.90015711],
       [-9.04492404,  1.10172574, 58.1960898 , ..., -8.16555486,
         3.26771302, -3.40711312],
       ...,
       [ 8.87583137, -6.69551931, -8.16555486, ..., 62.61216466,
        -1.25606695,  5.56410057],
       [ 6.88528597, -2.99318777,  3.26771302, ..., -1.25606695,
        17.72336005, -2.0619736 ],
       [ 3.20230281, -0.90015711, -3.40711312, ...,  5.56410057,
        -2.0619736 , 23.86693631]])

### Gaussian Kernel

Get the vector of squared norms

In [22]:
S = np.linalg.norm(D, axis = 1)
S = S * S

Set variance

In [23]:
variance = 10000

Compute Gaussian Kernel

In [24]:
gaussian_kernel = np.exp( ((2 * np.matmul(D, D.T)) - S - S[:,None]) / (2*variance) )
gaussian_kernel

array([[1.        , 0.99801584, 0.99501972, ..., 0.99658439, 0.99862488,
        0.99795056],
       [0.99801584, 1.        , 0.9965773 , ..., 0.9955807 , 0.99818722,
        0.99808953],
       [0.99501972, 0.9965773 , 1.        , ..., 0.99316649, 0.99653681,
        0.995566  ],
       ...,
       [0.99658439, 0.9955807 , 0.99316649, ..., 1.        , 0.99586618,
        0.99623954],
       [0.99862488, 0.99818722, 0.99653681, ..., 0.99586618, 1.        ,
        0.9977169 ],
       [0.99795056, 0.99808953, 0.995566  , ..., 0.99623954, 0.9977169 ,
        1.        ]])

In [48]:
def dual_SVM(D, kernel, C, epsilon):
    # augmented kernel matrix
    kernel_augmented = kernel + 1
    # make vector of step sizes
    eta = []
    for i in range(len(kernel)):
        val = 1 / kernel[i][i]
        eta.append(val)
    eta = np.array(eta)
    # initialize alpha vector
    t = 0
    alpha = np.random.uniform(0, 1, len(kernel))
    old_alpha = alpha.copy()
    while(True):
        # iterate thru n in random order
        r = list(range(1000))
        random.shuffle(r)
        for i in r:
            # first calculate the gradient
            # do the summation
            summation = 0
            for j in range(len(kernel)):
                summation += alpha[j] * y_train[j] * kernel[j][i]
            
            # multiply the gradient by the step size
            t = eta[i] * (1 - (y_train[i] * summation))
            print("eta", eta[i])
            # update alpha in place
            alpha[i] = alpha[i] + t
            # finally clamp alpha between 0 and C
            if alpha[i] < 0:
                alpha[i] = 0
            if alpha[i] > C:
                alpha[i] = C
        
        t += 1
        if np.linalg.norm(alpha - old_alpha) < epsilon:
            break
        
        old_alpha = alpha.copy()
        print(np.linalg.norm(alpha - old_alpha))

In [49]:
a = dual_SVM(D, linear_kernel, 10, 0.001)

eta 0.09863934600424021
eta 0.020480691997189288
eta 0.03418317195080185
eta 0.036265838948104645
eta 0.020902915894524542
eta 0.030344866920028836
eta 0.02192204372656588
eta 0.0670069929504128
eta 0.007746547936349672
eta 0.07389158212974788
eta 0.04809683823224539
eta 0.06660299931759835
eta 0.06346701468196413
eta 0.0661312184800988
eta 0.11152366103252108
eta 0.06653632397813052
eta 0.07249774663887032
eta 0.052667574373649086
eta 0.04979943277319322
eta 0.04367054592064368
eta 0.04671147046139123
eta 0.04941795855885662
eta 0.09317750368768972
eta 0.008198236809189777
eta 0.045385608670755646
eta 0.045082225378741694
eta 0.024027475889536572
eta 0.043156216360101154
eta 0.03370878064994905
eta 0.024596324358975158
eta 0.04625472351458114
eta 0.02921146835344512
eta 0.025776762371290708
eta 0.028867311291433072
eta 0.08895512017948391
eta 0.055840182603931636
eta 0.02619580786265312
eta 0.02586949530067255
eta 0.015929091167133005
eta 0.043799881628339574
eta 0.07739048509559592
e

eta 0.028367846379328685
eta 0.09006629578582988
eta 0.052115199230204426
eta 0.05386644796843578
eta 0.03142025466179445
eta 0.034366256718621715
eta 0.06268458196213716
eta 0.06170929274068087
eta 0.048399135971808525
eta 0.04550120576833954
eta 0.008065238398709892
eta 0.05611901452322736
eta 0.05785704263873593
eta 0.021138764947933
eta 0.052626729557523705
eta 0.03687063962616348
eta 0.0633502004562878
eta 0.028702419530192185
eta 0.0980166661913738
eta 0.04295133958670816
eta 0.019319666148103678
eta 0.052733077386597105
eta 0.04642159675825538
eta 0.00851641917358213
eta 0.04463320752037927
eta 0.05616668901449045
eta 0.03621612686593623
eta 0.06556144661897054
eta 0.041698049447036215
eta 0.05899195425981329
eta 0.026056829814424223
eta 0.053274966991749904
eta 0.03184382747222934
eta 0.059506635036957634
eta 0.023989677661823222
eta 0.06156106724427385
eta 0.03788894554187233
eta 0.052018195760379624
eta 0.06622509556775462
eta 0.024482410852714662
eta 0.03769621508504068
eta 

eta 0.06436677799295709
eta 0.04739860444936668
eta 0.13757053254133642
eta 0.0383932003851032
eta 0.05222747453590752
eta 0.010354892648939611
eta 0.09317750368768972
eta 0.06528800218439922
eta 0.06071628062250837
eta 0.1208305095841017
eta 0.08895512017948391
eta 0.056495233136063513
eta 0.06838871160190016
eta 0.053983147473572676
eta 0.04073533535758739
eta 0.12879393346286583
eta 0.04363925209702527
eta 0.05526941734442668
eta 0.05341434730235795
eta 0.03401895768481271
eta 0.06276694765372343
eta 0.022570927725412044
eta 0.10639884184196215
eta 0.04617719535845801
eta 0.04524265093454343
eta 0.043706605513991297
eta 0.047548138997854685
eta 0.0651683619580787
eta 0.030563263824349627
eta 0.052625734271255144
eta 0.06777451199778183
eta 0.10187432026454298
eta 0.05316881989172648
eta 0.059749115333452237
eta 0.03941725799646009
eta 0.050026113170289724
eta 0.07024876426700394
eta 0.04234745072116741
eta 0.05165751912343573
eta 0.014630815851975256
eta 0.071967574784293
eta 0.0667

eta 0.04939504076447784
eta 0.028120178365088257
eta 0.03193128560003138
eta 0.03522275157492383
eta 0.058872909724035666
eta 0.06556961459642228
eta 0.030281517617946327
eta 0.0634429695341979
eta 0.049587979527825184
eta 0.06653632397813052
eta 0.03251748154581742
eta 0.06543590795702568
eta 0.07564762059417147
eta 0.047971282319002766
eta 0.03519022936924588
eta 0.08792631421806513
eta 0.056005451651043155
eta 0.007746547936349672
eta 0.0746608502046494
eta 0.01745038263366404
eta 0.03424705800863837
eta 0.06194051926919452
eta 0.08451433556757719
eta 0.0670069929504128
eta 0.07938674210393121
eta 0.0759316652313847
eta 0.03769621508504068
eta 0.04824470354860159
eta 0.014626697631241399
eta 0.050187189491251366
eta 0.10009223694117085
eta 0.15950024532948434
eta 0.04862027656558328
eta 0.05322673300978734
eta 0.04343906206618591
eta 0.04179554244119579
eta 0.01402099669160458
eta 0.05892582357299474
eta 0.053392511738980145
eta 0.03621612686593623
eta 0.05642270974464227
eta 0.0704

eta 0.05500542243359665
eta 0.05899195425981329
eta 0.03026807715813343
eta 0.04222890408712785
eta 0.040041083180023515
eta 0.082417282384227
eta 0.04452485324692416
eta 0.038591020474202306
eta 0.019210372238292393
eta 0.07159094410095071
eta 0.04441313439907121
eta 0.09929311813638511
eta 0.07045568872072532
eta 0.04340306490862802
eta 0.021400717806635744
eta 0.007856021743845257
eta 0.06710912463649625
eta 0.0372179792376852
eta 0.01027219598922228
eta 0.10187432026454298
eta 0.04628139554306457
eta 0.08228829397129908
eta 0.02192204372656588
eta 0.03184382747222934
eta 0.023593371336735693
eta 0.030563263824349627
eta 0.0384211916119014
eta 0.03604503876664112
eta 0.049079825226503476
eta 0.051921215281845026
eta 0.046973986812619335
eta 0.06276694765372343
eta 0.036265838948104645
eta 0.021175380620252527
eta 0.03978917389772734
eta 0.007683716507641814
eta 0.021661757680302793
eta 0.0517320197631866
eta 0.07024876426700394
eta 0.050187189491251366
eta 0.03874469322510367
eta 0.

eta 0.00979712540801288
eta 0.03593426683324582
eta 0.07615426080150091
eta 0.06849696302109673
eta 0.058734378464665685
eta 0.05978678458787648
eta 0.04028974293839168
eta 0.0640086688105373
eta 0.053801375224848395
eta 0.008296694936319677
eta 0.0825685995200999
eta 0.014184989177481368
eta 0.05408811784223652
eta 0.056005451651043155
eta 0.028734461307172263
eta 0.03421230355343634
eta 0.06370671864857211
eta 0.05943295355805251
eta 0.05616668901449045
eta 0.02150878994912828
eta 0.0503244784499698
eta 0.018402534704462438
eta 0.12359406065186886
eta 0.06622509556775462
eta 0.05615548020680253
eta 0.05544709684137573
eta 0.05400466067763583
eta 0.0670069929504128
eta 0.04597242088540978
eta 0.07739048509559592
eta 0.034914349257609396
eta 0.05863159589793009
eta 0.055570462719240835
eta 0.09605715872408115
eta 0.008111417828847318
eta 0.09839509984073819
eta 0.056826569986099355
eta 0.049617072386810394
eta 0.04086332334122248
eta 0.03261818269934083
eta 0.019823984204689272
eta 0.0

eta 0.0418631522664154
eta 0.007395507691303355
eta 0.06655364820077088
eta 0.04625342493977874
eta 0.03026807715813343
eta 0.055840182603931636
eta 0.036763276148470886
eta 0.02535739208438431
eta 0.04600736139401569
eta 0.029596476215279603
eta 0.05309220544516494
eta 0.04719462939979976
eta 0.06971475058106166
eta 0.05578811778080292
eta 0.028527918788575742
eta 0.0661312184800988
eta 0.020851307363844628
eta 0.07811434877840108
eta 0.071967574784293
eta 0.06528800218439922
eta 0.05943295355805251
eta 0.06355620587601543
eta 0.05975533757027327
eta 0.03134950485915856
eta 0.04895968428701582
eta 0.08417835476417661
eta 0.0391516434839067
eta 0.08541628858173048
eta 0.10464825082135099
eta 0.0557967706118087
eta 0.07743057413913924
eta 0.059749115333452237
eta 0.06653632397813052
eta 0.038309231147735594
eta 0.04412770492462443
eta 0.021400717806635744
eta 0.040700381263693214
eta 0.023549270545145815
eta 0.06976729959098585
eta 0.08175303612332553
eta 0.08453945081157131
eta 0.04028

eta 0.031247279868540815
eta 0.028867311291433072
eta 0.03994255041624534
eta 0.04642159675825538
eta 0.05173033880181567
eta 0.007746547936349672
eta 0.032305754448252746
eta 0.047021378640764444
eta 0.08031146556155724
eta 0.036334029075607274
eta 0.05131364644277638
eta 0.024596324358975158
eta 0.09061173488316059
eta 0.06710912463649625
eta 0.043706605513991297
eta 0.029216613980568296
eta 0.07725947257197047
eta 0.008957819334878263
eta 0.056100579549453355
eta 0.04179554244119579
eta 0.03193128560003138
eta 0.0826305141119204
eta 0.0762652588298001
eta 0.008296694936319677
eta 0.07611255861552106
eta 0.04343906206618591
eta 0.034914349257609396
eta 0.043571941038943754
eta 0.03142025466179445
eta 0.057912065443250504
eta 0.0665259161628117
eta 0.028702419530192185
eta 0.05868208482284283
eta 0.06736729466199222
eta 0.040291196222809085
eta 0.025014844557521305
eta 0.06189431190959329
eta 0.06064355954950534
eta 0.07858543043623578
eta 0.049617072386810394
eta 0.06316966639707358


eta 0.02605858960689567
eta 0.03730532385496082
eta 0.05023891649217715
eta 0.04277197928839973
eta 0.03340560584017915
eta 0.023666648427909227
eta 0.0659671933646273
eta 0.032471226590470574
eta 0.018296016934110274
eta 0.051921215281845026
eta 0.06565694976514383
eta 0.01429546624024734
eta 0.06316966639707358
eta 0.021295257969089532
eta 0.024805440469834142
eta 0.04934679544940063
eta 0.021400717806635744
eta 0.02501100889465345
eta 0.059506635036957634
eta 0.025087838980614167
eta 0.0382810863770316
eta 0.04961015121118581
eta 0.05165751912343573
eta 0.05978678458787648
eta 0.05112941831110635
eta 0.03251748154581742
eta 0.019367334931087655
eta 0.12209730510588022
eta 0.11295226782517513
eta 0.038309231147735594
eta 0.12879393346286583
eta 0.007832991839138942
eta 0.03923257308496728
eta 0.0753860046652668
eta 0.07932675556293962
eta 0.046973986812619335
eta 0.0490899928366719
eta 0.010354892648939611
eta 0.007836778421835151
eta 0.08535311001889558
eta 0.037352540914029786
eta 

KeyboardInterrupt: 